In [2]:
import os,sys
sys.path.append('../..')

import numpy as np
import pandas as pd
from caproto.server import PVGroup, pvproperty, run
import nest_asyncio
nest_asyncio.apply()

import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s|%(name)s|%(message)s', datefmt='%H:%M:%S')

import pybeamtools
from pybeamtools.controls.devices import RealisticMagnet, GenericOutput
from pybeamtools.controls.virtual_tools import VirtualIOC, VirtualBeamline, VirtualDevice, VirtualOutput
from pybeamtools.controls.accelerator import SimController
from pybeamtools.controls.test_problems import Quadratic

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'VirtualIOC' from 'pybeamtools.controls.virtual_tools' (c:\googledrive\work\acc_code\pybeamtools\pybeamtools\controls\virtual_tools.py)

In [4]:
variables = ['AI:X0:AO', 'AI:X1:AO']
objectives = ['AI:OBJ0:output']
obj_idx_map = {o:i for i,o in enumerate(objectives)}

In [5]:
mag1 = RealisticMagnet(value=0, low=-2, high=2, noise=None, resolution=None, model='instant')
mag2 = RealisticMagnet(value=0, low=-2, high=2, noise=None, resolution=None, model='instant')

controller = SimController(variables, objectives)
def eval_fn(name):
    return controller.read(name)
output = GenericOutput(eval_fn, objectives[0])

In [6]:
print(mag1.read_exact(),mag1.write(-1),mag1.read_exact(),mag1.write(0),mag1.read_exact(),mag1.read())

0 True -1 True 0 0


In [7]:
class VirtualBeamline(PVGroup):
    m0 = SubGroup(VirtualIOC, device=mag1, prefix='X0:')
    m1 = SubGroup(VirtualIOC, device=mag2, prefix='X1:')
    obj1 = SubGroup(VirtualOutput, device=output, output_name='OBJ0', prefix='OBJ0:')

bl = VirtualBeamline(prefix='AI:')
print('PVs:', list(bl.pvdb))

10:37:32|caproto.server.server|class VirtualBeamline subgroup attr m0: <caproto.server.server.SubGroup object at 0x0000026524D1B1C0>
10:37:32|caproto.server.server|class VirtualBeamline subgroup attr m1: <caproto.server.server.SubGroup object at 0x0000026524D1B430>
10:37:32|caproto.server.server|class VirtualBeamline subgroup attr obj1: <caproto.server.server.SubGroup object at 0x0000026524D1B400>
10:37:32|pybeamtools.controls.virtual_tools|VO called with args=() kwargs={'prefix': 'AI:OBJ0:', 'macros': {}, 'parent': <__main__.VirtualBeamline object at 0x0000026524D1BEE0>, 'name': 'VirtualBeamline.obj1'}


PVs: ['AI:X0:AI', 'AI:X0:AO', 'AI:X1:AI', 'AI:X1:AO', 'AI:OBJ0:output']


In [8]:
bl.m1.ao.name

'VirtualBeamline.m1.ao'

In [13]:
problem = Quadratic(n_var=len(variables))

last_inputs = None
last_results = None
def get_objectives(obj): 
    #print('getobj1')
    global last_results
    global last_inputs
    assert obj in objectives
    # gather inputs
    inputs = np.array([bl.pvdb[var].value for var in variables])
    inputs = inputs[None,:]
    assert inputs.shape == (1,len(variables))
    if last_inputs is not None and np.array_equal(inputs,last_inputs):
        results = last_results.copy()
    else:
        results = problem._evaluate(inputs)
        #print(results)
        assert results.shape == (1,len(objectives))
        last_inputs = inputs.copy()
        last_results = results
    val = results[0, obj_idx_map[obj]]
    logger.info(f'Requested value of {obj} @ {inputs} = {val}')
    return val

controller.eval_fn = get_objectives

In [14]:
# import torch
# model = torch.jit.load('/home/helios/NKUKLEV/ox/daily/linac_nn/model_scripted.pt')
# model.eval()
#
# X_nominal = torch.tensor([[-0.2750,  1.4518,  1.9890, -0.3003, -1.1654,  0.3311,  1.1663,  1.1860,
#           0.6450, -0.3071, -0.5011,  0.5966,  0.0162, -0.2055,  1.0583,  0.2522]],device='cuda:0')
#
# last_inputs = None
# last_results = None
# idx1 = 3
# idx2 = 2
# def get_objectives(obj):
#     #print('getobj1')
#     global last_results
#     global last_inputs
#     assert obj in objectives
#     # gather inputs
#     inputs = np.array([bl.pvdb[var].value for var in variables])
#     inputs = inputs[None,:]
#     assert inputs.shape == (1,len(variables))
#     if last_inputs is not None and np.array_equal(inputs,last_inputs):
#         results = last_results.copy()
#     else:
#         X_test = X_nominal.clone()
#         X_test[0, idx1] = inputs[0,0]
#         X_test[0, idx2] = inputs[0,1]
#         with torch.no_grad():
#             pred = model(X_test)
#         out = pred.cpu().numpy()[0,1]
#         results = out#problem._evaluate(inputs)
#         print(results)
#         results = np.array([results])[None,:]
#         assert results.shape == (1,len(objectives))
#         last_inputs = inputs.copy()
#         last_results = results
#     val = results[0, obj_idx_map[obj]]
#     logger.info(f'Requested value of {obj} @ {inputs} = {val}')
#     return val
#
# controller.eval_fn = get_objectives

In [15]:
get_objectives('AI:OBJ0:output')

10:38:03|pybeamtools.controls.virtual_tools|Requested value of AI:OBJ0:output @ [[0. 0.]] = 0.08000000000000003


0.08000000000000003

In [16]:
run(bl.pvdb, log_pv_names=True, interfaces=['0.0.0.0'])

10:38:05|caproto.ctx|EPICS_CA_SERVER_PORT set to 5064. This is the UDP port to be used for searches, and the first TCP server port to be tried.
10:38:05|caproto.ctx|Asyncio server starting up...
10:38:05|caproto.ctx|Listening on 0.0.0.0:5064
10:38:05|caproto.ctx|UDP socket bound on 0.0.0.0:5064
10:38:05|caproto.ctx|Calling startup method 'AI:X0:AO.server_startup'
10:38:05|caproto.ctx|Calling startup method 'AI:X1:AO.server_startup'
10:38:05|caproto.ctx|Server startup complete.
10:38:05|caproto.ctx|PVs available:
AI:X0:AI
AI:X0:AO
AI:X1:AI
AI:X1:AO
AI:OBJ0:output
10:38:05|caproto.ctx|Will send beacons to ['255.255.255.255:5065']
10:38:05|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=0, address='0.0.0.0')


ao_startup: instance=<caproto.server.server.PvpropertyDouble object at 0x0000026524A84D60> async_lib=<caproto.asyncio.server.AsyncioAsyncLayer object at 0x0000026524E50CD0>
ao_startup: instance=<caproto.server.server.PvpropertyDouble object at 0x0000026524A488E0> async_lib=<caproto.asyncio.server.AsyncioAsyncLayer object at 0x0000026524E50CD0>


10:38:05|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=1, address='0.0.0.0')
10:38:06|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=2, address='0.0.0.0')
10:38:06|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=3, address='0.0.0.0')
10:38:06|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=4, address='0.0.0.0')
10:38:07|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=5, address='0.0.0.0')
10:38:08|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=6, address='0.0.0.0')
10:38:11|caproto.bcast|Beacon(version=13, server_port=5064, beacon_id=7, address='0.0.0.0')
10:38:14|caproto.bcast|VersionRequest(priority=0, version=13)
10:38:14|caproto.bcast|SearchRequest(name='AI:OBJ0:output', cid=43851, version=13, reply=5)
10:38:14|caproto.bcast|VersionResponse(version=13)
10:38:14|caproto.bcast.search|SearchResponse(port=5064, ip='255.255.255.255', cid=43851, version=13)
10:38:14|caproto.bcast|VersionRequest(priority=0, 

RuntimeError: Event loop stopped before Future completed.